## Graph Modeling
When modeling you data in a graph database, it's generally beneficial to follow a structure that allows you to effectively query and leverage the relationshipps within the data.
1. Understand the domain and define specific use cases (questions) for the application
2. Develop the initial graph data model
    - Model the nodes (entities)
    - Model the relationships between nodes
3. Test the use cases against the initial data model.
4. Create the graph (instance model) with test data using Cypher.
5. Test the use cases, including performance against the graph.
6. Refactor (improve) the graph data model due to a change in the key use cases or for performance reasons.
7. Implement the refactring on the graph and retest using Cypher.

## Domain Knowledge
1. Identify the stakeholders and developers of the application.
2. With the stakeholders and developers:
    - Describe the application in detail.
    - Identify the users of the applicaiton (people, systems).
    - Agree upon the use cases for the application.
    - Rank the importance of the use cases.

## Two types of Model:
1. Data Model:A data model is an abstract model that organizes data elements and standardizes how they relate to one another and to the properties of real-world entities.

2. Instance Model: An instance model, on the other hand, refers to the actual data or database instance that adheres to the structure of the **Data Model**.

We Create an **Instance Model** after creating **Data Model** because we need to confirm that our data model satisfies the use cases of our application.

**Abstract vs. Concrete**: A **Data Model** is abstract, focusing on the blueprint and design, whereas an **Instance Model** is concrete, representing actual data at a particular time.

## Model Relationships:
1. The relationship name should be all captials.
2. The start and end label must have a label associated to them.

## QUERY Example:
1. What role did a person play in the movie?
    - Retrieve the name of the person.
    - Follow the ACTED_IN relationship to movies
    - Filter the movie by its title.
    - Return the role from the ACTED_IN relationship.



## Basic Example Questions - 
1. What was the revenue of Apple in the first quarter of 2024?
2. What are the risk factors associated with Adobe?
3. How were the net sales of Amazon in 2024 compared to 2023?
4. What was the percentage difference in total revenues of Netflix between 2024 and 2023, which year seen more sales.?

**BASED ON THESE BASIC QUESTIONS IT WOULD BE BETTER TO MODEL GRAPH IN THE FOLLLOWING STRUCTURE:**

NODES:
- Filing: accession, form
- ParentChunk: attributes - content, sub_heading_dict, is_table, chunk_id
- ChildChunk: attributes - content, sub_heading_dict
- Company: attributes - name
- Heading: attributes - name

RELATIONSHIPS:
- HAS_CHILD
- FILED: attributes - filing_date
- HAS_SECTION
- HAS_HEADING

PATTERN
- (Company)-[:FILED {date}]->(Filing)
- (Filing)-[:HAS_SECTION]->(ParentChunk)
- (ParentChunk)-[:HAS_CHILD]->(ChildChunk)
- (ParentChunk)-[:HAS_HEADING]->(Heading)

NOTE - To avoid passing very large parent sections to the LLM, sibling relationships make it easy to retrieve a limited context window (top-K related children) without loading the entire parent chunk.